# Fetch Articles Abstracts from Scopus Search API
API KEY from Scopus, get it from https://dev.elsevier.com

In [ ]:
API_KEY = "" #Private API KEY

In [ ]:
#max number of results to fetch each time depending of Service Level in Scopus API 
COUNT=5

Your query in here

In [ ]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [2]:
import requests
import json
import pandas as pd

### Search request to Scopus API

In [ ]:
def run_scopus_search(start):
    resp = requests.get("http://api.elsevier.com/content/search/scopus?start=%s&count=%s&query=%s" % (str(start),str(COUNT),QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    print("fetching Scopus IDs... %d " % start)
    results= json.loads(resp.text.encode('utf-8'))
    return results

### Return scopus IDs from results 

In [ ]:
def get_scopus_ids(results):
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

### Return total numbers of articles found 

In [ ]:
def get_totalResults(results):
    return int(results['search-results']['opensearch:totalResults'])

### Return basic info about article

In [ ]:
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

### Return abstract article

In [ ]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

because fetching results could take hours we store results in base file in case the process is interrupt, you can start it again without fetch again the abstracts

In [ ]:
from pathlib import Path
number_articles_allready_load=0
base = Path("./data/scopus_search_base.csv")
base_exist=False
new_base_exist=False
if base.is_file():
    article_set_readed=pd.read_csv("./data/scopus_search_base.csv")
    base_exist=True
    number_articles_allready_load=article_set_readed.shape[0]
    print('base exists ,%d articles '% number_articles_allready_load) 
new_base_exist=base_exist

### Main

run scopus search, return top 200 and number of total articles
loop by next 200 until reach total articles

write dataset to csv file

In [ ]:
start=0
article_data=[]

n=0

results=run_scopus_search(start)
totalResults=get_totalResults(results)

scopus_ids=get_scopus_ids(results)
if base_exist==True:
    scopus_idsdf=pd.DataFrame(scopus_ids)
    scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values

n_ids_fetched =len(scopus_ids)
print("number of scopus ids fetched : %d de %d" % (n_ids_fetched,totalResults))

while (n<totalResults-number_articles_allready_load):
    nn=0
    for scopus_id in scopus_ids:

        results_info = get_scopus_info(scopus_id[0])
        results_text = get_scopus_text(scopus_id[0])
        try:
            title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
            pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
            try:
                abstract=str(results_text['full-text-retrieval-response']['coredata']['dc:description'])
                abstract=abstract.replace('Abstract\n               \n                  ','')
            except:
                abstract='(not available)'
            print("fetching %d / %d..." % (n+1,totalResults)) 
            n=n+1
            nn=nn+1
            author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
            article_data.append({'Scopus_ID' : scopus_id[0],'title':title,'pubDate':pubDate,'abstract': abstract,'author': author[0][0]})
        except:
            pass
    if nn>0:
        article_set=pd.DataFrame(article_data)
        article_data=[]
        if (new_base_exist==False):
            print('creating base')
            article_set.to_csv("./data/scopus_search_base.csv",index=False)
            new_base_exist=True
        else:
            print('append to base')
            article_set.to_csv("./data/scopus_search_base.csv",mode="a",header=False,index=False)
           
    start=start+n_ids_fetched
    results=run_scopus_search(start)
    scopus_ids=get_scopus_ids(results)
    n_ids_fetched =len(scopus_ids)
    if base_exist==True:
        scopus_idsdf=pd.DataFrame(scopus_ids)
        scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values
        print('novos artigos= %d %d' % (len(scopus_ids),article_set_readed.shape[0]))
    print("number of scopus ids fetched : %d de %d" % (n_ids_fetched+start,totalResults))
print("Finish!")

In [3]:
article_set=pd.read_csv("./data/scopus_search_base.csv") 
print(article_set.info)
print(article_set.head)

<bound method DataFrame.info of                   Scopus_ID  \
0     SCOPUS_ID:85035128299   
1     SCOPUS_ID:85038847134   
2     SCOPUS_ID:85036467319   
3     SCOPUS_ID:85038816447   
4     SCOPUS_ID:85034053217   
...                     ...   
1253  SCOPUS_ID:84978641606   
1254  SCOPUS_ID:84978903927   
1255  SCOPUS_ID:84961115864   
1256  SCOPUS_ID:84976330384   
1257  SCOPUS_ID:84929121341   

                                               abstract           author  \
0     This study develops a stochastic approach to t...           Lee M.   
1                                       (not available)       Sevlian R.   
2     Cloud computing has drastically transformed th...         Kumar J.   
3     Online reviews have been extensively studied i...            Ma Y.   
4     In this paper a new hybrid method combining va...          Naik J.   
...                                                 ...              ...   
1253                                    (not available)       P

In [4]:
article_set

,Scopus_ID,abstract,author,pubDate,title
0,SCOPUS_ID:85035128299,This study develops a stochastic approach to t...,Lee M.,2018-06-01,Modeling and forecasting hotel room demand bas...
1,SCOPUS_ID:85038847134,(not available),Sevlian R.,2018-06-01,A scaling law for short term load forecasting ...
2,SCOPUS_ID:85036467319,Cloud computing has drastically transformed th...,Kumar J.,2018-04-01,Workload prediction in cloud using artificial ...
3,SCOPUS_ID:85038816447,Online reviews have been extensively studied i...,Ma Y.,2018-04-01,Effects of user-provided photos on hotel revie...
4,SCOPUS_ID:85034053217,In this paper a new hybrid method combining va...,Naik J.,2018-04-01,Short term wind power forecasting using hybrid...
...,...,...,...,...,...
1253,SCOPUS_ID:84978641606,(not available),Pedraza L.,2016-07-01,Evaluation of nonlinear forecasts for radioele...
1254,SCOPUS_ID:84978903927,(not available),Bogaraj T.,2016-07-01,Intelligent energy management control for inde...
1255,SCOPUS_ID:84961115864,The progressive application of Information and...,Gamarra C.,2016-07-01,A knowledge discovery in databases approach fo...
1256,SCOPUS_ID:84976330384,(not available),Afilal M.,2016-07-01,Forecasting the Emergency Department Patients ...
